<a href="https://colab.research.google.com/github/sashaTribe/legalDocNLP/blob/main/Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#from tmtoolkit.corpus import Corpus
#from tmtoolkit.corpus import corpus_add_files
import string
import re
#corp = Corpus(language='en')
#corpus_add_files(corp, '1/smallNum/case0.txt')
import matplotlib.pyplot as plt
import nltk
import nltk.data
from nltk.corpus import reuters
from nltk import FreqDist
from nltk.corpus import stopwords
from nltk import ngrams
nltk.download('reuters')
import os
nltk.download('stopwords')
sw_nltk = stopwords.words('english')
print(sw_nltk)
import numpy as np
import random as rd
import pandas as pd 
import re
from sklearn import linear_model
from sklearn.decomposition import PCA
import gensim
from gensim import corpora, models, similarities
from gensim.parsing.preprocessing import STOPWORDS
import spacy
from gensim.models import Word2Vec
from gensim.models import KeyedVectors
from gensim.models.phrases import Phraser, Phrases

[nltk_data] Downloading package reuters to /root/nltk_data...
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

In [ ]:

common_words = ["of", "with","without", "in", "the", "for", "to","and","or","a"]
stop_words = []
for i in range(len(sw_nltk)):
    if sw_nltk[i] not in common_words:
        stop_words.append(sw_nltk[i])
print(stop_words)
def filter_line(line):
    split_array = line.split()
    punc = '''!()-[]{};:'"\,<>./?@#$%^&*_~'''
    j = 0
    while j < len(split_array):
        for letter in split_array[j]:
            if letter in punc:
                split_array[j] = split_array[j].replace(letter, " ")
                
        j += 1
    
    word_sentence = ' '.join(split_array)
    split_array = word_sentence.split(' ')
    #split_array[:] = (word for word in split_array if word not in sw_nltk)
    i = 0
    while i < len(split_array):
        if split_array[i] in stop_words:
            split_array.remove(split_array[i])   
        else:   
            i += 1
    return split_array

def filter_legal_file (filename):
    words = []
    with open(filename, encoding='utf-8') as f:
        lines = f.readlines()
        for line in lines:
            words.append(filter_line(line))
    return words


['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'an', 'but', 'if', 'because', 'as', 'until', 'while', 'at', 'by', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'from', 'up', 'down', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', 'too', 'very', 's', 't', 'can', 'will', 'just', 'don', "don

In [ ]:
word_to_index = dict() #need variable to convert word to an integer value
index_to_word = dict() #need variable to convert integer value to word
labels_index = {}
index_list = [] #label ids
label_text = [] #list of label texts
TOPIC_DATA = "drive/MyDrive/Topics"
#common_words = ["of", "with","without", "in", "the", "for", "to"]
corpus = []
vocab_size = 0
count = 0
texts = []
for name in os.listdir(TOPIC_DATA):
    path = os.path.join(TOPIC_DATA,name)
    if os.path.isdir(path):
        label_id = len(labels_index)
        labels_index[name]  = label_id
        for fname in os.listdir(path):
            fpath = os.path.join(path,fname)
            texts.append(filter_legal_file(fpath))
            index_list.append(fname)

all_sentences = []
for text in texts:
  all_sentences += text   

#gathers phrases from list of sentences that includes words in common words array:
print(len(all_sentences))
phrases = Phrases(all_sentences,connector_words = common_words)
bigram = Phraser(phrases)
all_sentences = list(bigram[all_sentences])

index_to_word = gensim.corpora.Dictionary(all_sentences)
index_to_word.filter_extremes(no_below=15)
index_to_word.compactify()
print(index_to_word.iterkeys())
#voc_size = len(word_to_index)
len_of_corpus = len(corpus)



"""print(vocab_size)
print(word_to_index)
print(len(index_list))
print(len_of_corpus)
corpus = [word_to_index.doc2bow(text) for text in bigram"""

18705


'print(vocab_size)\nprint(word_to_index)\nprint(len(index_list))\nprint(len_of_corpus)\ncorpus = [word_to_index.doc2bow(text) for text in bigram'

In [ ]:
vocab = model.corpus_count

UnicodeDecodeError: ignored

In [ ]:
from gensim.matutils import corpus2csc
import multiprocessing
model = Word2Vec(all_sentences,min_count=15,vector_size=200,batch_words=3,workers=multiprocessing.cpu_count())
model.save("word2vec.model")
corpus= list(model.wv.index_to_key)
print(len(corpus))
print(corpus)





8111
['the', 'of', 'to', 'a', 'in', 'and', 'for', 'or', 'with', 'court', 'district_court', 'see', 'would', 'may', 'case', 'cir', 'claim', 'id', 'claims', 'state', 'plaintiffs', 'also', 'united_states', 'one', 'law', 'must', 'act', 'whether', 'order', 'defendants', 'first', 'issue', 'could', 'b', 'evidence', 'federal', 'rule', 'statute', 'however', 'decision', 'judgment', 'plaintiff', 'action', 'government', 'time', 'two', 'parties', 'even', 'thus', 'without', 'defendant', 'complaint', 'appeal', 'filed', 'fact', 'section', 'made', 'congress', 'based', 'cases', 'use', 'trial', 'part', 'property', 'question', 'review', 'argument', 'courts', 'found', 'held', 'required', 'agreement', 'motion', 'supreme_court', 'second', 'patent', 'interest', 'therefore', 'rather', 'right', 'contract', 'see_also', 'jurisdiction', 'including', 'class', 'states', 'party', 'provision', 'conduct', 'c', 'finding', 'agency', 'although', 'bank', 'within', 'make', 'debtor', 'information', 'new', 'used', 'record', 'a

In [ ]:
text_vectors = [model.wv.get_mean_vector(word) for word in corpus]
df = pd.DataFrame(text_vectors)
print(df.shape)
df.index = corpus
df.head(30)

(8111, 200)


,0,1,2,3,4,5,6,7,8,9,...,190,191,192,193,194,195,196,197,198,199
the,0.020638,0.053145,0.007389,0.075910,-0.024175,0.081055,0.128321,0.165704,-0.074899,-0.078401,...,0.076845,-0.017503,-0.123773,0.021660,-0.008601,-0.112191,-0.032693,-0.020425,0.041689,-0.006505
of,-0.051160,0.066571,0.006547,0.094604,-0.027324,0.104509,0.128503,0.124037,-0.075559,-0.020866,...,0.074847,0.047659,-0.107903,0.003703,0.053354,-0.109571,-0.049076,-0.011938,0.048491,-0.066907
to,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
a,0.020805,-0.034761,0.074568,-0.015899,0.076272,-0.053174,0.128870,0.043779,0.016874,-0.102750,...,-0.005565,-0.112126,-0.077113,0.003932,-0.003384,-0.093455,-0.040843,0.096186,-0.008109,0.046414
in,-0.025228,0.031241,0.036296,0.018322,0.016358,0.073784,0.133388,0.154112,-0.061823,-0.014747,...,0.044965,0.059714,-0.079731,-0.000772,0.085467,-0.077808,-0.071527,-0.013685,-0.005327,-0.018023
and,-0.002211,-0.001760,0.055432,0.001212,0.046315,0.010305,0.131129,0.098945,-0.022474,-0.058748,...,0.019700,-0.026206,-0.078422,0.001580,0.041041,-0.085631,-0.056185,0.041251,-0.006718,0.014196
for,0.011541,0.037911,0.048345,0.030627,-0.007525,0.082717,0.101075,0.181126,-0.083388,-0.017832,...,0.117707,0.015627,-0.095689,-0.004362,0.065523,-0.069279,-0.064348,-0.031288,0.004950,-0.047240
or,0.074241,0.009250,0.090143,-0.033350,0.012275,0.060925,0.073646,0.238215,-0.091217,-0.014798,...,0.160568,-0.016405,-0.083476,-0.012428,0.077691,-0.028987,-0.079620,-0.050638,-0.038591,-0.027573
with,0.062883,0.036087,0.073354,0.026133,-0.016020,0.072463,0.091056,0.217600,-0.091077,-0.032799,...,0.143451,-0.014187,-0.081830,-0.007298,0.048780,-0.064309,-0.052146,-0.075567,-0.050197,-0.025454
court,0.038330,0.041130,0.048106,0.025729,-0.003085,0.061395,0.090779,0.196103,-0.085103,-0.041149,...,0.105555,-0.020066,-0.095328,0.007805,0.047448,-0.058393,-0.070358,-0.023619,0.002250,-0.008016


In [ ]:
#compute the correlation matrix
X_correlation = df.corr()
eigen_values,eigen_vectors = np.linalg.eig(X_correlation)
args = (-eigen_values).argsort()
eigen_values = eigen_vectors[args]
eigen_vectors = eigen_vectors[:, args]


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
model = Word2Vec.load("word2vec.model")
model["justice"]

TypeError: ignored

In [ ]:
X = model.wv

df=pd.DataFrame(df)
df.shape
df.head()

,0,1,2,3,4,5,6,7,8,9,...,190,191,192,193,194,195,196,197,198,199
0,0.164282,-0.006067,1.031472,-0.298445,-0.649184,-0.556603,0.140197,0.272902,-1.382528,0.185008,...,-0.092183,0.181610,-0.284186,-0.197008,0.632631,-0.437057,-0.222788,-0.251496,-0.457002,0.010221
1,-0.065180,0.169813,0.254845,-0.279989,0.368263,0.356420,-0.484635,0.216406,-0.515198,0.634090,...,0.101991,0.275110,0.188581,0.292253,1.250111,-0.216484,0.842536,-0.679669,-0.372505,-0.944796
2,0.196790,-0.087474,-0.724577,-1.240630,0.278146,0.702787,0.130676,0.159990,-0.276826,0.222383,...,1.031175,0.544629,-0.787271,-0.213854,0.241202,0.566802,-0.367100,-0.904064,0.494189,-0.413505
3,0.269884,0.198082,0.458050,-0.028738,-0.034390,-0.009797,0.327739,1.101743,-0.161945,-0.558617,...,-0.553682,-0.532857,-0.396849,-0.415335,-0.091610,-0.748141,-0.156989,-0.115452,0.496064,0.336284
4,0.434134,0.418704,0.763415,0.277046,0.919312,-0.171539,0.442421,-0.183112,-0.761748,0.564472,...,0.055617,0.332562,0.026377,0.991116,0.594323,-0.645701,-1.202511,1.033372,0.279379,0.234472


UnicodeDecodeError: ignored

In [ ]:
for name in os.listdir(TOPIC_DATA):
    path = os.path.join(TOPIC_DATA,name)
    print(path)
    if os.path.isdir(path):
        index_list.append(name)
        if(path=='drive/MyDrive/Topics/1'):
          for fname in os.listdir(path):
              #print(fname)
              index_list.append(fname)
              fpath = os.path.join(path,fname)
              text.append(filter_legal_file(fpath))
for row in text:
  for i in range(len(row)):
    for word in row[i]:
      corpus.append(word) 
print(corpus[1])

drive/MyDrive/Topics/1
drive/MyDrive/Topics/10
drive/MyDrive/Topics/11
drive/MyDrive/Topics/12
drive/MyDrive/Topics/13
drive/MyDrive/Topics/2
drive/MyDrive/Topics/3
drive/MyDrive/Topics/4
drive/MyDrive/Topics/5
drive/MyDrive/Topics/6
drive/MyDrive/Topics/7
drive/MyDrive/Topics/8
drive/MyDrive/Topics/9
consideration
